# State Farm
* [State Farm Distracted Driver Detection | Kaggle](https://www.kaggle.com/c/state-farm-distracted-driver-detection/data)
* [courses/statefarm.ipynb at master · fastai/courses](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/statefarm.ipynb)

In [1]:
%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"

import os, sys
sys.path.insert(1, os.path.join(sys.path[0], 'utils'))
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


## Setup Bach Data

In [2]:
batch_size = 64
path = "/home/ubuntu/datasets/state-farm-distracted-driver-detection/"

In [3]:
batches = get_batches(path+'train', batch_size=batch_size)

Found 18424 images belonging to 10 classes.


In [ ]:
conv_val_feat = load_array(path+'results/conv_val_feat.dat')

In [ ]:
test_batches = get_batches(path+'test', batch_size=batch_size)

## training

In [ ]:
vgg = Vgg16()
model=vgg.model

In [ ]:
[i for i,l in enumerate(model.layers) if type(l) is Convolution2D]

In [ ]:
# 最後のConvolutionレイヤのID取得
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]

In [ ]:
# Convolutionレイヤから後ろをとってしまう
conv_layers = model.layers[:last_conv_idx+1]

In [ ]:
conv_layers

In [ ]:
# Listなので、Sequentialモデルで包む
conv_model = Sequential(conv_layers)
conv_model.summary()

### Create more batches(x3)

In [4]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 18424 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [ ]:
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [ ]:
conv_feat.shape

In [ ]:
conv_val_feat.shape

In [ ]:
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

In [ ]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [ ]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_feat.shape

In [ ]:
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_val_feat.shape

### Use Data Augmentation

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

In [ ]:
# 2倍のtrain モデルからpredictをする
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*4)
save_array(path+'results/da_conv_feat5.dat', da_conv_feat)

In [ ]:
da_conv_feat = load_array(path+'results/da_conv_feat4.dat')
da_conv_feat.shape

In [ ]:
# オリジナルと合体させて、6倍！ラベルも6倍
# 合体させるとメモリエラー !!
# http://forums.fast.ai/t/statefarm-kaggle-comp/183/89
#da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

In [ ]:
da_trn_labels = np.concatenate([trn_labels]*2)

## Build Model

In [ ]:
conv_layers[-1].output_shape[1:]

In [5]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=(512, 14, 14)),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [6]:
p=0.8

In [7]:
bn_model = Sequential(get_bn_da_layers(p))
bn_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_1 (MaxPooling2D)    (None, 512, 7, 7)     0           maxpooling2d_input_1[0][0]       
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 25088)         0           maxpooling2d_1[0][0]             
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 25088)         0           flatten_1[0][0]                  
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 256)           6422784     dropout_1[0][0]                  
___________________________________________________________________________________________

In [8]:
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.0001
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.save_weights(path+'models/conv4.h5')

In [9]:
bn_model.load_weights(path+'models/conv4.h5')

## Pseudo labeling

predict_generaterで生成したデータからpredict
Convolutionの最後で分割したところから、再度予測を実施。

In [ ]:
val_pseudo = bn_model.predict(conv_val_feat, batch_size=batch_size)

In [ ]:
val_pseudo.shape

pseudoラベルとデータを元のデータに結合。これでどのくらいデータ量が増えるのだろうか

In [ ]:
comb_pseudo = np.concatenate([da_trn_labels, val_pseudo])
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

In [ ]:
(comb_pseudo.shape, comb_feat.shape)

In [ ]:
bn_model.load_weights(path+'models/conv4.h5')

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.0001
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=10, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.save_weights(path+'models/bn-ps4.h5')

In [18]:
bn_model.load_weights(path+'models/bn-ps4.h5')

## test data

In [10]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [11]:
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [19]:
preds = bn_model.predict(conv_test_feat, batch_size=batch_size*2)

In [20]:
subm = do_clip(preds,0.93)

In [21]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.167224,0.093129,0.039857,0.118063,0.077593,0.219514,0.076944,0.037321,0.082709,0.087646
1,img_14887.jpg,0.049100,0.073142,0.033573,0.022011,0.031941,0.061064,0.049071,0.444175,0.141332,0.094592
2,img_62885.jpg,0.126713,0.062846,0.045519,0.153162,0.216708,0.082628,0.100528,0.035037,0.112392,0.064467
3,img_45125.jpg,0.180964,0.111499,0.036491,0.061702,0.047212,0.068203,0.067346,0.051007,0.195276,0.180300
4,img_22633.jpg,0.144262,0.144127,0.063668,0.100404,0.072613,0.076692,0.129445,0.065303,0.122506,0.080980


In [22]:
submission.to_csv('subm.gz', index=False, compression='gzip')

In [23]:
FileLink('subm.gz')

/home/ubuntu/nbs/subm.gz